# MariaDB Vector Magics - Test Notebook

This notebook is used for testing and development of MariaDB Vector Magics functionality.

**Note:** This is a development notebook. For production examples, see `demo_notebook.ipynb`.

## Setup

In [16]:
from mariadb_vector_magics.magics.main import MariaDBVectorMagics
from IPython import get_ipython

ip = get_ipython()
magic_instance = MariaDBVectorMagics(ip)
ip.register_magics(magic_instance)

print("✅ Magic commands registered directly")



✅ Magic commands registered directly


In [17]:
%lsmagic


Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %code_wrap  %colors  %conda  %config  %connect_info  %connect_mariadb  %cp  %create_vector_store  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %get_secret  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_embedding_model  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %mamba  %man  %matplotlib  %micromamba  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %query_table  %quickref  %rag_query  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %semantic_search  %set_env  %set_secret  %show_vector_tables  %store  %subshell  %sx  %system  %tb  %time  %timeit  %unalias  %unloa

In [19]:
# Load the extension
%reload_ext mariadb_vector_magics

In [20]:
# Connect to test database (replace with your test credentials)
%connect_mariadb --password=yourpassword --database=vectordb

'Connected successfully'

## Model Loading Tests

In [21]:
# Test embedding model loading
%load_embedding_model all-MiniLM-L6-v2

'Model all-MiniLM-L6-v2 loaded'

## Vector Store Tests

In [22]:
# Test vector store creation
%create_vector_store --table test_documents --dimensions 384 --distance cosine --drop_if_exists

'Table test_documents created'

In [23]:
# Test table listing
%show_vector_tables

['test_documents']

## Document Embedding Tests

In [24]:
%%embed_table --table test_documents --batch_size 8 --chunk_size 200
[
    "MariaDB is an open-source relational database management system.",
    "Vector search enables semantic similarity matching.",
    "Jupyter notebooks provide an interactive computing environment.",
    "Python is a versatile programming language for AI.",
    "The HNSW algorithm provides efficient approximate nearest neighbor search."
]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Inserted 5 records into test_documents'

## Query Tests

In [23]:
# Test table querying
%query_table test_documents

ID,Text Preview
1,MariaDB is an open-source relational database management system.
2,Vector search enables semantic similarity matching.
3,Jupyter notebooks provide an interactive computing environment.
4,Python is a versatile programming language for AI.
5,The HNSW algorithm provides efficient approximate nearest neighbor search.


{'name': 'test_documents',
 'row_count': 10,
 'structure': [('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
  ('document_text', 'longtext', 'NO', '', None, ''),
  ('metadata', 'longtext', 'YES', '', None, ''),
  ('embedding', 'vector(384)', 'NO', 'MUL', None, ''),
  ('created_at', 'timestamp', 'YES', '', 'current_timestamp()', ''),
  ('updated_at',
   'timestamp',
   'YES',
   '',
   'current_timestamp()',
   'on update current_timestamp()')],
 'sample_data': [(1,
   'MariaDB is an open-source relational database management system.'),
  (2, 'Vector search enables semantic similarity matching.'),
  (3, 'Jupyter notebooks provide an interactive computing environment.'),
  (4, 'Python is a versatile programming language for AI.'),
  (5,
   'The HNSW algorithm provides efficient approximate nearest neighbor search.')]}

## Semantic Search Tests

In [24]:
# Test basic semantic search
%semantic_search "database technology" --table test_documents --top_k 3

[(1,
  'MariaDB is an open-source relational database management system.',
  '{"original_doc_index": 0, "chunk_index": 0, "is_chunk": false}'),
 (6,
  'MariaDB is an open-source relational database management system.',
  '{"original_doc_index": 0, "chunk_index": 0, "is_chunk": false}'),
 (3,
  'Jupyter notebooks provide an interactive computing environment.',
  '{"original_doc_index": 2, "chunk_index": 2, "is_chunk": false}')]

In [25]:
# Test semantic search with distance scores
%semantic_search "artificial intelligence" --table test_documents --top_k 5 --show_distance

[(4,
  'Python is a versatile programming language for AI.',
  '{"original_doc_index": 3, "chunk_index": 3, "is_chunk": false}',
  0.4698034516865438),
 (9,
  'Python is a versatile programming language for AI.',
  '{"original_doc_index": 3, "chunk_index": 3, "is_chunk": false}',
  0.4698034516865438),
 (7,
  'Vector search enables semantic similarity matching.',
  '{"original_doc_index": 1, "chunk_index": 1, "is_chunk": false}',
  0.7359286169359396),
 (2,
  'Vector search enables semantic similarity matching.',
  '{"original_doc_index": 1, "chunk_index": 1, "is_chunk": false}',
  0.7359286169359396),
 (8,
  'Jupyter notebooks provide an interactive computing environment.',
  '{"original_doc_index": 2, "chunk_index": 2, "is_chunk": false}',
  0.7390737295562808)]

## RAG Tests

**Note:** Set GROQ_API_KEY environment variable for these tests to work.

In [ ]:
# Set API key for testing (use environment variable in production)
import os
# Uncomment and set your API key for testing
# os.environ['GROQ_API_KEY'] = ''

In [28]:
# Test RAG query (only works if API key is set)
%rag_query "What is vector search used for?" --table test_documents --top_k 3

{'question': 'What is vector search used for?',
 'answer': 'Vector search is used for semantic similarity matching.',
 'context': ['Vector search enables semantic similarity matching.',
  'Vector search enables semantic similarity matching.',
  'The HNSW algorithm provides efficient approximate nearest neighbor search.'],
 'context_metadata': [{'id': 7,
   'distance': 0.39767342077272483,
   'metadata': '{"original_doc_index": 1, "chunk_index": 1, "is_chunk": false}'},
  {'id': 2,
   'distance': 0.39767342077272483,
   'metadata': '{"original_doc_index": 1, "chunk_index": 1, "is_chunk": false}'},
  {'id': 10,
   'distance': 0.6311283087110092,
   'metadata': '{"original_doc_index": 4, "chunk_index": 4, "is_chunk": false}'}],
 'model': 'llama-3.3-70b-versatile',
 'temperature': 0.2,
 'retrieval_time': 0.34799695014953613,
 'generation_time': 1.6451878547668457,
 'total_time': 1.993190050125122,
 'context_count': 3}

## Cleanup

Clean up test data after testing.

In [ ]:
# Uncomment to clean up test data
# %create_vector_store --table test_documents --dimensions 384 --drop_if_exists